## 计算QM9数据集中分子在x、y、z三个轴上的最大直径

In [1]:
import os
import torch
import numpy as np
from tqdm import tqdm

In [4]:
def calculate_molecule_diameter(coords, atoms_channel, padding_index=999):
    """
    计算单个分子在x、y、z三个轴上的直径
    
    Args:
        coords: 分子坐标 (N, 3)
        atoms_channel: 原子类型 (N,)
        padding_index: 填充索引，用于忽略无效原子
    
    Returns:
        dict: 包含x、y、z轴直径的字典
    """
    # 过滤掉填充的原子
    valid_mask = atoms_channel != padding_index
    valid_coords = coords[valid_mask]
    
    if len(valid_coords) == 0:
        return {"x": 0.0, "y": 0.0, "z": 0.0}
    
    # 计算每个轴上的最大和最小坐标
    min_coords = torch.min(valid_coords, dim=0).values
    max_coords = torch.max(valid_coords, dim=0).values
    
    # 计算直径（最大值 - 最小值）
    diameters = max_coords - min_coords
    
    return {
        "x": diameters[0].item(),
        "y": diameters[1].item(), 
        "z": diameters[2].item()
    }


In [5]:
def analyze_qm9_diameters(data_dir="dataset/data", split="train", max_samples=None):
    """
    分析QM9数据集中所有分子的直径
    
    Args:
        data_dir: 数据目录
        split: 数据集分割 ("train", "val", "test")
        max_samples: 最大分析样本数，None表示分析全部
    """
    # 加载数据
    data_path = os.path.join(data_dir, "qm9", f"{split}_data.pth")
    if not os.path.exists(data_path):
        print(f"错误：找不到数据文件 {data_path}")
        return None
    
    print(f"正在加载 {split} 数据集...")
    data = torch.load(data_path, weights_only=False)
    
    if max_samples:
        data = data[:max_samples]
        print(f"只分析前 {max_samples} 个分子")
    
    print(f"开始分析 {len(data)} 个分子...")
    
    # 存储所有分子的直径
    all_diameters = []
    
    # 遍历所有分子
    for i, sample in enumerate(tqdm(data, desc=f"分析{split}分子")):
        coords = sample["coords"]
        atoms_channel = sample["atoms_channel"]
        
        # 计算当前分子的直径
        diameters = calculate_molecule_diameter(coords, atoms_channel)
        all_diameters.append(diameters)
    
    # 转换为numpy数组便于分析
    diameters_array = np.array([[d["x"], d["y"], d["z"]] for d in all_diameters])
    
    # 计算统计信息
    print(f"\n=== QM9 {split} 数据集分子直径统计 ===")
    print(f"分析分子数量: {len(all_diameters)}")
    
    for axis in ["x", "y", "z"]:
        axis_idx = {"x": 0, "y": 1, "z": 2}[axis]
        axis_diameters = diameters_array[:, axis_idx]
        
        print(f"\n{axis.upper()} 轴:")
        print(f"  最小值: {axis_diameters.min():.4f}")
        print(f"  最大值: {axis_diameters.max():.4f}")
        print(f"  平均值: {axis_diameters.mean():.4f}")
        print(f"  中位数: {np.median(axis_diameters):.4f}")
        print(f"  标准差: {axis_diameters.std():.4f}")
    
    # 计算总体最大直径（三个轴中的最大值）
    max_diameters = np.max(diameters_array, axis=1)
    print(f"\n总体最大直径（三个轴中的最大值）:")
    print(f"  最小值: {max_diameters.min():.4f}")
    print(f"  最大值: {max_diameters.max():.4f}")
    print(f"  平均值: {max_diameters.mean():.4f}")
    print(f"  中位数: {np.median(max_diameters):.4f}")
    print(f"  标准差: {max_diameters.std():.4f}")
    
    # 找到最大直径的分子
    max_idx = np.argmax(max_diameters)
    print(f"\n最大直径分子 (索引 {max_idx}):")
    print(f"  X轴直径: {diameters_array[max_idx, 0]:.4f}")
    print(f"  Y轴直径: {diameters_array[max_idx, 1]:.4f}")
    print(f"  Z轴直径: {diameters_array[max_idx, 2]:.4f}")
    print(f"  最大直径: {max_diameters[max_idx]:.4f}")
    
    return diameters_array, max_diameters


In [8]:
print("开始分析QM9数据集分子直径...")
    
# 分析所有数据集分割
all_results = {}
    
for split in ["train", "val", "test"]:
    print(f"\n{'='*50}")
    print(f"分析 {split} 数据集")
    print(f"{'='*50}")
        
    # 分析完整数据集
    result = analyze_qm9_diameters(split=split)
    if result is not None:
        all_results[split] = result
    
# 汇总统计
print(f"\n{'='*50}")
print("汇总统计")
print(f"{'='*50}")
    
all_max_diameters = []
for split, (diameters_array, max_diameters) in all_results.items():
    all_max_diameters.extend(max_diameters)
    print(f"{split} 数据集最大直径范围: {max_diameters.min():.4f} - {max_diameters.max():.4f}")
    
all_max_diameters = np.array(all_max_diameters)
print(f"\n整个QM9数据集:")
print(f"  总分子数: {len(all_max_diameters)}")
print(f"  最大直径范围: {all_max_diameters.min():.4f} - {all_max_diameters.max():.4f}")
print(f"  平均最大直径: {all_max_diameters.mean():.4f}")
print(f"  中位数最大直径: {np.median(all_max_diameters):.4f}")
print(f"  95%分位数: {np.percentile(all_max_diameters, 95):.4f}")
print(f"  99%分位数: {np.percentile(all_max_diameters, 99):.4f}")

开始分析QM9数据集分子直径...

分析 train 数据集
正在加载 train 数据集...
开始分析 97734 个分子...


分析train分子: 100%|██████████| 97734/97734 [00:06<00:00, 15947.73it/s]



=== QM9 train 数据集分子直径统计 ===
分析分子数量: 97734

X 轴:
  最小值: 0.0301
  最大值: 10.3183
  平均值: 4.4916
  中位数: 4.4511
  标准差: 1.0942

Y 轴:
  最小值: 0.0000
  最大值: 11.4391
  平均值: 5.2739
  中位数: 5.1587
  标准差: 1.0931

Z 轴:
  最小值: 0.0000
  最大值: 10.0368
  平均值: 4.1983
  中位数: 4.2699
  标准差: 1.2966

总体最大直径（三个轴中的最大值）:
  最小值: 1.4361
  最大值: 11.4391
  平均值: 5.7999
  中位数: 5.6865
  标准差: 0.8798

最大直径分子 (索引 39224):
  X轴直径: 2.8513
  Y轴直径: 11.4391
  Z轴直径: 2.2934
  最大直径: 11.4391

分析 val 数据集
正在加载 val 数据集...
开始分析 20042 个分子...


分析val分子: 100%|██████████| 20042/20042 [00:01<00:00, 14118.27it/s]



=== QM9 val 数据集分子直径统计 ===
分析分子数量: 20042

X 轴:
  最小值: 0.0771
  最大值: 10.0219
  平均值: 4.4899
  中位数: 4.4541
  标准差: 1.0819

Y 轴:
  最小值: 0.0000
  最大值: 11.8271
  平均值: 5.2704
  中位数: 5.1611
  标准差: 1.0957

Z 轴:
  最小值: 0.0000
  最大值: 9.7145
  平均值: 4.1964
  中位数: 4.2616
  标准差: 1.2984

总体最大直径（三个轴中的最大值）:
  最小值: 1.8741
  最大值: 11.8271
  平均值: 5.7975
  中位数: 5.6784
  标准差: 0.8757

最大直径分子 (索引 7845):
  X轴直径: 1.5421
  Y轴直径: 11.8271
  Z轴直径: 1.7709
  最大直径: 11.8271

分析 test 数据集
正在加载 test 数据集...
开始分析 13055 个分子...


分析test分子: 100%|██████████| 13055/13055 [00:01<00:00, 11576.75it/s]



=== QM9 test 数据集分子直径统计 ===
分析分子数量: 13055

X 轴:
  最小值: 0.0425
  最大值: 9.8766
  平均值: 4.4796
  中位数: 4.4416
  标准差: 1.0918

Y 轴:
  最小值: 1.2802
  最大值: 11.1337
  平均值: 5.2723
  中位数: 5.1423
  标准差: 1.0931

Z 轴:
  最小值: 0.0069
  最大值: 9.8686
  平均值: 4.2028
  中位数: 4.2726
  标准差: 1.2779

总体最大直径（三个轴中的最大值）:
  最小值: 1.2802
  最大值: 11.1337
  平均值: 5.7958
  中位数: 5.6787
  标准差: 0.8744

最大直径分子 (索引 9534):
  X轴直径: 2.2680
  Y轴直径: 11.1337
  Z轴直径: 1.8350
  最大直径: 11.1337

汇总统计
train 数据集最大直径范围: 1.4361 - 11.4391
val 数据集最大直径范围: 1.8741 - 11.8271
test 数据集最大直径范围: 1.2802 - 11.1337

整个QM9数据集:
  总分子数: 130831
  最大直径范围: 1.2802 - 11.8271
  平均最大直径: 5.7991
  中位数最大直径: 5.6841
  95%分位数: 7.4748
  99%分位数: 8.4610
